BITCOIN:¶
Using Coingecko API to create candlestick graph for Bitcoin.

We will use the API to get the price data for 30 days with 24 observation per day, 1 per hour.

We will find the max, min, open, and close price per day meaning we will have 30 candlesticks and use that to generate the candlestick graph.

Although we are using the CoinGecko API we will use a Python client/wrapper for the API called PyCoinGecko.

PyCoinGecko will make performing the requests easy and it will deal with the enpoint targeting.

In [7]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance
!pip install nbformat
!pip install --upgrade nbformat


In [8]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc
import nbformat

Retrieving the Data.¶
Using the get_coin_market_chart_by_id(id, vs_currency, days).

id is the name of the coin you want,

vs_currency is the currency you want the price in,

days is how many days back from today you want.

In [9]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=30)

In [10]:
type(bitcoin_data)

dict

In [11]:
data = pd.DataFrame(bitcoin_data)
data.head()

,prices,market_caps,total_volumes
0,"[1620676964675, 56010.8323693779]","[1620676964675, 1047748040121.421]","[1620676964675, 70779739753.5695]"
1,"[1620680581083, 55240.32123935162]","[1620680581083, 1033336497167.5593]","[1620680581083, 73125501113.53194]"
2,"[1620684096840, 55994.192668268144]","[1620684096840, 1047440974641.4423]","[1620684096840, 77007537203.23456]"
3,"[1620687768554, 56274.44427494624]","[1620687768554, 1052685905372.6512]","[1620687768554, 78031062256.16512]"
4,"[1620691322924, 55901.66759456145]","[1620691322924, 1045714370754.1615]","[1620691322924, 76672978747.69038]"


The response we get is in the form of a JSON which includes the price, market caps, and total volumes along with timestamps for each observation.

We are focused on the prices so we will select that data.

In [12]:
bitcoin_price_data = bitcoin_data['prices']

bitcoin_price_data[0:5]

[[1620676964675, 56010.8323693779],
 [1620680581083, 55240.32123935162],
 [1620684096840, 55994.192668268144],
 [1620687768554, 56274.44427494624],
 [1620691322924, 55901.66759456145]]

In [13]:
type(bitcoin_price_data)

list

Turning this data into a Pandas DataFrame.

In [14]:
data = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])

In [15]:
type(data)

pandas.core.frame.DataFrame

Now that we have the DataFrame we will convert the timestamp to datetime and save it as a column called Date. We will map our unix_to_datetime to each timestamp and convert it to a readable datetime.

In [16]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))

apply() function calls the lambda function and applies it to every row or column of the dataframe and returns a modified copy of the dataframe.

So here we are making a new column called date, from the timestamp column, by applying date.time function and
 dividing every row and column by 1000

In [17]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})

Using this modified dataset we can now group by the Date and find the min, max, open, and close for the candlesticks.

Finally we are now ready to use plotly to create our Candlestick Chart.

In [20]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])
fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()